<a href="https://colab.research.google.com/github/EloizioHMD/Python_DS/blob/main/Limpeza_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Em continuação o preparação da base de dados, vamos fazer a limpeza da base de dados arrumando algumas incosistências antes do partir para o ML.

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/rec_imovel_02.csv')

Carregados os dados direto do Google Drive, vamos fazer algumas explorações.

In [4]:
df.shape

(85608, 9)

In [5]:
df = df.drop_duplicates()

In [6]:
df.shape

(42804, 9)

Os dados estavam duplicados. Em seguida vamos verificar valores faltantes.

In [7]:
pd.DataFrame(data = [
    df.isna().sum()/df.shape[0]*100,
    df.isnull().sum()/df.shape[0]*100],
    index=['Qtd. NA (%)','Qtd. Null (%)']
    )

,Unnamed: 0,endereco,bairro,area_m2,quartos,banheiros,vagas,valor,condominio
Qtd. NA (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Qtd. Null (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Não há valores faltantes, mas imagino que seja necessário alguns ajustes.

In [8]:
df.head()

,Unnamed: 0,endereco,bairro,area_m2,quartos,banheiros,vagas,valor,condominio
0,0,Rua General Polidoro,Várzea,55,2,2,1,330000,350
1,1,Rua Monte Castelo,Boa Vista,33,1 Quarto,1 Banheiro,1,Sob Consulta,n
2,2,Rua Setúbal,Boa Viagem,86,3,3,2 s,780000,740
3,3,Rua Quarenta e Oito,Aflitos,69,3,3,1,378000,690
4,4,Rua Esmeraldino Bandeira,Graças,54,1 Quarto,1 Banheiro,1,240000,590


Dropar coluna `Unnamed: 0` que não faz sentido.

In [40]:
# Dropar a coluna unnamed
df = df.drop(columns='Unnamed: 0')

É Preciso verificar string presente no campo `valor`:

In [32]:
df['valor'].value_counts()

430000                                 4679
450000                                 3065
330000                                 2378
380000                                 1209
310000                                 1189
499999                                 1189
305000                                 1189
1110000                                1189
510000                                 1189
498000                                 1189
167000                                 1189
499000                                 1189
100000                                 1189
535000                                 1189
299999      Preço abaixo do mercado    1189
378000                                 1189
240000                                 1189
690000                                 1189
320000                                 1189
520000                                 1189
400000      Preço abaixo do mercado    1189
165000                                 1189
380000      Preço abaixo do merc

In [33]:
df.loc[df['valor'] == '299999      Preço abaixo do mercado', 'valor'] = 300000

In [34]:
df.loc[df['valor'] == '400000      Preço abaixo do mercado', 'valor'] = 400000

In [35]:
df.loc[df['valor'] == '380000      Preço abaixo do mercado', 'valor'] = 380000

In [36]:
df.loc[df['valor'] == '490000      Preço abaixo do mercado', 'valor'] = 490000

In [9]:
df = df.drop(df[df['valor'] == 'Sob Consulta'].index) # apagar as linhas que contém valor 'Sob Consulta

In [37]:
df['valor'].value_counts()

430000     4679
450000     3065
330000     2378
380000     1209
310000     1189
499999     1189
305000     1189
1110000    1189
510000     1189
498000     1189
167000     1189
499000     1189
100000     1189
535000     1189
300000     1189
378000     1189
240000     1189
690000     1189
320000     1189
520000     1189
400000     1189
165000     1189
380000     1189
265000     1188
575000     1169
780000     1169
900000     1169
600000      490
250000       52
1050000      25
350000       21
490000       20
480000        7
220000        1
Name: valor, dtype: int64

Vamos verificar os valores do Quartos:

In [10]:
df['quartos'].value_counts()

2             16606
3             15478
4              4775
1   Quarto     2378
Name: quartos, dtype: int64

In [11]:
df.loc[df['quartos'] == '1   Quarto', 'quartos'] = 1

In [12]:
df['quartos'].value_counts()

2    16606
3    15478
4     4775
1     2378
Name: quartos, dtype: int64

Vamos verificar os valores de Banheiros:

In [13]:
df['banheiros'].value_counts()

2               19070
3               15411
1   Banheiro     3567
4                1189
Name: banheiros, dtype: int64

In [14]:
df.loc[df['banheiros'] == '1   Banheiro', 'banheiros'] = 1

In [15]:
df['banheiros'].value_counts()

2    19070
3    15411
1     3567
4     1189
Name: banheiros, dtype: int64

Verificar os valores de vagas:

In [19]:
df['vagas'].value_counts()

1        25699
2   s    13534
--           4
Name: vagas, dtype: int64

In [20]:
df.loc[df['vagas'] == '2   s', 'vagas'] = 2

Quando a vagas com valor `--` é mais interessante dropar a linha, pois ante os resultados será um outlier.

In [21]:
df = df.drop(df[df['vagas'] == '--'].index)

In [25]:
df['vagas'].value_counts()

1    25699
2    13534
Name: vagas, dtype: int64

Ainda precisamos verificar os tipos e outras informações necessárias para realizarmos o modelo.

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39233 entries, 0 to 42803
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   endereco    39233 non-null  object
 1   bairro      39233 non-null  object
 2   area_m2     39233 non-null  int64 
 3   quartos     39233 non-null  object
 4   banheiros   39233 non-null  object
 5   vagas       39233 non-null  object
 6   valor       39233 non-null  object
 7   condominio  39233 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.7+ MB


In [69]:
df.endereco = df.endereco.astype(str)

In [70]:
df.bairro = df.bairro.astype(str)

In [71]:
df.quartos = df.quartos.astype(int)

In [76]:
df.banheiros = df.banheiros.astype(int)

In [73]:
df.vagas = df.vagas.astype(int)

In [74]:
df.valor  = df.valor.astype(int)

In [77]:
print(df.dtypes)

endereco      object
bairro        object
area_m2        int64
quartos        int64
banheiros      int64
vagas          int64
valor          int64
condominio    object
dtype: object


In [78]:
df

,endereco,bairro,area_m2,quartos,banheiros,vagas,valor,condominio
0,Rua General Polidoro,Várzea,55,2,2,1,330000,350
2,Rua Setúbal,Boa Viagem,86,3,3,2,780000,740
3,Rua Quarenta e Oito,Aflitos,69,3,3,1,378000,690
4,Rua Esmeraldino Bandeira,Graças,54,1,1,1,240000,590
5,Rua Edson Álvares,Casa Forte,94,2,3,2,535000,850
...,...,...,...,...,...,...,...,...
42799,Rua Visconde de Jequitinhonha,Boa Viagem,64,3,2,2,600000,n
42800,Rua do Riachuelo,Boa Vista,95,4,3,1,265000,555
42801,Rua Jonathas de Vasconcelos,Boa Viagem,55,2,2,1,305000,600
42802,Rua Leon Helmer,Boa Viagem,90,3,3,1,499999,850


In [84]:
df.to_csv('recimovel_db.csv', encoding = 'utf-8')